# BSP - Binocular sight point

This project is intended to calculate the position in which direction to look with binoculars. The exact position can be calculated based on the height to sea level, orientation to the north with an angle for horizontal (0-360 degrees) and with a tilt angle (up to 0-90 degrees) for vertical.

https://github.com/M-E-E-R-e-V/bsp

![bsp](bsp.png)



When you reproject to a projected coordinate system, the distance method is probably using a planar calculation. 5634, for instance, uses Lambert conformal conic and is centered in Europe so more distance distortion than the UTM zone. UTM isn't distance-preserving either and 3857/900913 is notoriously bad at distances. Look for a geodesic or in a pinch a great circle method; then the CRS doesn't matter.

## Geopy
https://geopy.readthedocs.io/en/stable/#module-geopy.distance

Great circle distances differ from geodesic distances by up to 0.5%

Geodesic distance always produces a more accurate result

In [1]:
import math
from geopy import distance

print("Geopy")

p1 = (28.067808, -17.314608, 300)
p2 = (28.04381 , -17.32907, 0)

flat_distance = distance.distance(p1[:2], p2[:2]).m

print('Flat:', flat_distance)

euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)

print('Euclidian (+height):', euclidian_distance)

geodesic_distance = distance.geodesic((p1[:2]),(p2[:2])).meters

print('Geodesic:', geodesic_distance)

print('Great Circle:', distance.great_circle(p1[:2], p2[:2]).meters)

Geopy
Flat: 3015.660773645268
Euclidian (+height): 3030.5461391806552
Geodesic: 3015.660773645268
Great Circle: 3022.3534766826824


## Shapely
### CRS

https://epsg.io/4083 REGCAN95 / UTM zone 28N

https://epsg.io/5634

https://epsg.io/900913 #Google Maps Global Mercator -- Spherical Mercator (unofficial - used in open source projects / OSGEO)

Adapt projection to the points.

I prefer transverse plate carrée over transverse Mercator, but over small enough areas ("large scale"), almost any projection "centered" near the region of interest will give good accuracy.

In [5]:
import geopandas as gpd
from shapely.geometry import LineString 
print("Shapely")

L_4326= gpd.GeoSeries([LineString ([(-17.314608 , 28.067808), (-17.32907, 28.04381)])], crs="EPSG:4326")

L1 = L_4326.to_crs(epsg=4083) #REGCAN95 / UTM zone 28N
L2 = L_4326.to_crs(epsg=5634)
L3 = L_4326.to_crs(epsg=900913) #Google Maps Global Mercator -- Spherical Mercator (unofficial - used in open source projects / OSGEO)

print('distance 4083:', L1.length[0])
print('distance 5634:', L2.length[0])
print('distance 900913:', L3.length[0])


Shapely
distance 4083: 3016.3930360833283
distance 5634: 3125.312697809095
distance 900913: 3428.6341730737054


## Pyproj vs Haversine

http://www.movable-type.co.uk/scripts/latlong.html

The UTM zone is easy to "work out" based on the longitudes. Pick a typical longitude lambda, -180 <= lambda < 180, and use it to compute the zone number as Int((180+lambda)/6)+1. Use the sign of the latitude to decide between north and south. 

In [7]:
from shapely.geometry import Point
from pyproj import Proj

point1_geo = (-17.314608, 28.067808)
point2_geo = (-17.32907, 28.04381)

coord= point1_geo[0]
utm_zone = int((180+coord)/6)+1
print("UTM :",utm_zone)

proj = Proj(proj='utm',zone=utm_zone,ellps='WGS84')
#proj = Proj(init="epsg:3785")  # spherical mercator, should work anywhere...

point1 = proj(point1_geo[0], point1_geo[1])
point2 = proj(point2_geo[0], point2_geo[1])

point1_cart = Point(point1)
point2_cart = Point(point2)

print ("p1-p2 (cartesian)", point1_cart.distance(point2_cart))

#https://www.onlineconversion.com/map_greatcircle_distance.htm
print("Great-circle distance between two points (haversine)", "3022 m")

UTM : 28
p1-p2 (cartesian) 3016.3930361324888
Great-circle distance between two points (haversine) 3022 m


Javascript

Using OL ol.sphere.getDistance method gives 3022 meters. This is the great circle distance between the points.

turf.js method turf.distance gives exactly the same result: 3022 meters.

Questions : 

https://gis.stackexchange.com/questions/478679/which-method-to-calculate-distance-from-coordinates

https://gis.stackexchange.com/questions/7293/measuring-distance-in-spherical-mercator-vs-zoned-utm?rq=1